AES is a symmetric encryption algorithm that is widely used for securing sensitive data.

AES operates on fixed-size blocks of data and supports three key sizes: 128 bits, 192 bits, and 256 bits. The algorithm consists of a series of rounds where the input plaintext is transformed using a combination of substitution, permutation, and mixing operations. These operations are applied based on the encryption key to produce the ciphertext.

#Implementation

There are a few modes of operations for AES, we will consider ECB mode(Electronic Codebook)

##Division into blocks of 16 bytes

In AES-128 ECB mode, the plaintext is divided into blocks of 128 bits (16 bytes)

In [85]:
def break_into_blocks(data):
    result = []
    for i in range(len(data)//16):
        interval_16 = data[16*i: 16*(i+1)]
        block = []
        for k in range(4):
            row = []
            for j in range(4):
                row.append(interval_16[k + 4*j])
            block.append(row)
        result.append(block)
    return result

##S-Box1

In AES S-BOX is a fixed lookup table that performs byte substitution operations.

The S-Box is a 16x16 matrix that maps each 8-bit input value to a corresponding 8-bit output value. It is constructed in such a way that each possible input byte has a unique output byte. The S-Box is designed to provide confusion and non-linearity in the AES algorithm, enhancing its resistance against cryptographic attacks.

In [86]:
sbox = [
  [int('63', 16), int('7c', 16), int('77', 16), int('7b', 16), int('f2', 16), int('6b', 16), int('6f', 16), int('c5', 16), int(
      '30', 16), int('01', 16), int('67', 16), int('2b', 16), int('fe', 16), int('d7', 16), int('ab', 16), int('76', 16)],
  [int('ca', 16), int('82', 16), int('c9', 16), int('7d', 16), int('fa', 16), int('59', 16), int('47', 16), int('f0', 16), int(
      'ad', 16), int('d4', 16), int('a2', 16), int('af', 16), int('9c', 16), int('a4', 16), int('72', 16), int('c0', 16)],
  [int('b7', 16), int('fd', 16), int('93', 16), int('26', 16), int('36', 16), int('3f', 16), int('f7', 16), int('cc', 16), int(
      '34', 16), int('a5', 16), int('e5', 16), int('f1', 16), int('71', 16), int('d8', 16), int('31', 16), int('15', 16)],
  [int('04', 16), int('c7', 16), int('23', 16), int('c3', 16), int('18', 16), int('96', 16), int('05', 16), int('9a', 16), int(
      '07', 16), int('12', 16), int('80', 16), int('e2', 16), int('eb', 16), int('27', 16), int('b2', 16), int('75', 16)],
  [int('09', 16), int('83', 16), int('2c', 16), int('1a', 16), int('1b', 16), int('6e', 16), int('5a', 16), int('a0', 16), int(
      '52', 16), int('3b', 16), int('d6', 16), int('b3', 16), int('29', 16), int('e3', 16), int('2f', 16), int('84', 16)],
  [int('53', 16), int('d1', 16), int('00', 16), int('ed', 16), int('20', 16), int('fc', 16), int('b1', 16), int('5b', 16), int(
      '6a', 16), int('cb', 16), int('be', 16), int('39', 16), int('4a', 16), int('4c', 16), int('58', 16), int('cf', 16)],
  [int('d0', 16), int('ef', 16), int('aa', 16), int('fb', 16), int('43', 16), int('4d', 16), int('33', 16), int('85', 16), int(
      '45', 16), int('f9', 16), int('02', 16), int('7f', 16), int('50', 16), int('3c', 16), int('9f', 16), int('a8', 16)],
  [int('51', 16), int('a3', 16), int('40', 16), int('8f', 16), int('92', 16), int('9d', 16), int('38', 16), int('f5', 16), int(
      'bc', 16), int('b6', 16), int('da', 16), int('21', 16), int('10', 16), int('ff', 16), int('f3', 16), int('d2', 16)],
  [int('cd', 16), int('0c', 16), int('13', 16), int('ec', 16), int('5f', 16), int('97', 16), int('44', 16), int('17', 16), int(
      'c4', 16), int('a7', 16), int('7e', 16), int('3d', 16), int('64', 16), int('5d', 16), int('19', 16), int('73', 16)],
  [int('60', 16), int('81', 16), int('4f', 16), int('dc', 16), int('22', 16), int('2a', 16), int('90', 16), int('88', 16), int(
      '46', 16), int('ee', 16), int('b8', 16), int('14', 16), int('de', 16), int('5e', 16), int('0b', 16), int('db', 16)],
  [int('e0', 16), int('32', 16), int('3a', 16), int('0a', 16), int('49', 16), int('06', 16), int('24', 16), int('5c', 16), int(
      'c2', 16), int('d3', 16), int('ac', 16), int('62', 16), int('91', 16), int('95', 16), int('e4', 16), int('79', 16)],
  [int('e7', 16), int('c8', 16), int('37', 16), int('6d', 16), int('8d', 16), int('d5', 16), int('4e', 16), int('a9', 16), int(
      '6c', 16), int('56', 16), int('f4', 16), int('ea', 16), int('65', 16), int('7a', 16), int('ae', 16), int('08', 16)],
  [int('ba', 16), int('78', 16), int('25', 16), int('2e', 16), int('1c', 16), int('a6', 16), int('b4', 16), int('c6', 16), int(
      'e8', 16), int('dd', 16), int('74', 16), int('1f', 16), int('4b', 16), int('bd', 16), int('8b', 16), int('8a', 16)],
  [int('70', 16), int('3e', 16), int('b5', 16), int('66', 16), int('48', 16), int('03', 16), int('f6', 16), int('0e', 16), int(
      '61', 16), int('35', 16), int('57', 16), int('b9', 16), int('86', 16), int('c1', 16), int('1d', 16), int('9e', 16)],
  [int('e1', 16), int('f8', 16), int('98', 16), int('11', 16), int('69', 16), int('d9', 16), int('8e', 16), int('94', 16), int(
      '9b', 16), int('1e', 16), int('87', 16), int('e9', 16), int('ce', 16), int('55', 16), int('28', 16), int('df', 16)],
  [int('8c', 16), int('a1', 16), int('89', 16), int('0d', 16), int('bf', 16), int('e6', 16), int('42', 16), int('68', 16), int(
      '41', 16), int('99', 16), int('2d', 16), int('0f', 16), int('b0', 16), int('54', 16), int('bb', 16), int('16', 16)]
]

In [87]:
reverse_sbox = [
    [int('52', 16), int('09', 16), int('6a', 16), int('d5', 16), int('30', 16), int('36', 16), int('a5', 16), int('38', 16), int(
        'bf', 16), int('40', 16), int('a3', 16), int('9e', 16), int('81', 16), int('f3', 16), int('d7', 16), int('fb', 16)],
    [int('7c', 16), int('e3', 16), int('39', 16), int('82', 16), int('9b', 16), int('2f', 16), int('ff', 16), int('87', 16), int(
        '34', 16), int('8e', 16), int('43', 16), int('44', 16), int('c4', 16), int('de', 16), int('e9', 16), int('cb', 16)],
    [int('54', 16), int('7b', 16), int('94', 16), int('32', 16), int('a6', 16), int('c2', 16), int('23', 16), int('3d', 16), int(
        'ee', 16), int('4c', 16), int('95', 16), int('0b', 16), int('42', 16), int('fa', 16), int('c3', 16), int('4e', 16)],
    [int('08', 16), int('2e', 16), int('a1', 16), int('66', 16), int('28', 16), int('d9', 16), int('24', 16), int('b2', 16), int(
        '76', 16), int('5b', 16), int('a2', 16), int('49', 16), int('6d', 16), int('8b', 16), int('d1', 16), int('25', 16)],
    [int('72', 16), int('f8', 16), int('f6', 16), int('64', 16), int('86', 16), int('68', 16), int('98', 16), int('16', 16), int(
        'd4', 16), int('a4', 16), int('5c', 16), int('cc', 16), int('5d', 16), int('65', 16), int('b6', 16), int('92', 16)],
    [int('6c', 16), int('70', 16), int('48', 16), int('50', 16), int('fd', 16), int('ed', 16), int('b9', 16), int('da', 16), int(
        '5e', 16), int('15', 16), int('46', 16), int('57', 16), int('a7', 16), int('8d', 16), int('9d', 16), int('84', 16)],
    [int('90', 16), int('d8', 16), int('ab', 16), int('00', 16), int('8c', 16), int('bc', 16), int('d3', 16), int('0a', 16), int(
        'f7', 16), int('e4', 16), int('58', 16), int('05', 16), int('b8', 16), int('b3', 16), int('45', 16), int('06', 16)],
    [int('d0', 16), int('2c', 16), int('1e', 16), int('8f', 16), int('ca', 16), int('3f', 16), int('0f', 16), int('02', 16), int(
        'c1', 16), int('af', 16), int('bd', 16), int('03', 16), int('01', 16), int('13', 16), int('8a', 16), int('6b', 16)],
    [int('3a', 16), int('91', 16), int('11', 16), int('41', 16), int('4f', 16), int('67', 16), int('dc', 16), int('ea', 16), int(
        '97', 16), int('f2', 16), int('cf', 16), int('ce', 16), int('f0', 16), int('b4', 16), int('e6', 16), int('73', 16)],
    [int('96', 16), int('ac', 16), int('74', 16), int('22', 16), int('e7', 16), int('ad', 16), int('35', 16), int('85', 16), int(
        'e2', 16), int('f9', 16), int('37', 16), int('e8', 16), int('1c', 16), int('75', 16), int('df', 16), int('6e', 16)],
    [int('47', 16), int('f1', 16), int('1a', 16), int('71', 16), int('1d', 16), int('29', 16), int('c5', 16), int('89', 16), int(
        '6f', 16), int('b7', 16), int('62', 16), int('0e', 16), int('aa', 16), int('18', 16), int('be', 16), int('1b', 16)],
    [int('fc', 16), int('56', 16), int('3e', 16), int('4b', 16), int('c6', 16), int('d2', 16), int('79', 16), int('20', 16), int(
        '9a', 16), int('db', 16), int('c0', 16), int('fe', 16), int('78', 16), int('cd', 16), int('5a', 16), int('f4', 16)],
    [int('1f', 16), int('dd', 16), int('a8', 16), int('33', 16), int('88', 16), int('07', 16), int('c7', 16), int('31', 16), int(
        'b1', 16), int('12', 16), int('10', 16), int('59', 16), int('27', 16), int('80', 16), int('ec', 16), int('5f', 16)],
    [int('60', 16), int('51', 16), int('7f', 16), int('a9', 16), int('19', 16), int('b5', 16), int('4a', 16), int('0d', 16), int(
        '2d', 16), int('e5', 16), int('7a', 16), int('9f', 16), int('93', 16), int('c9', 16), int('9c', 16), int('ef', 16)],
    [int('a0', 16), int('e0', 16), int('3b', 16), int('4d', 16), int('ae', 16), int('2a', 16), int('f5', 16), int('b0', 16), int(
        'c8', 16), int('eb', 16), int('bb', 16), int('3c', 16), int('83', 16), int('53', 16), int('99', 16), int('61', 16)],
    [int('17', 16), int('2b', 16), int('04', 16), int('7e', 16), int('ba', 16), int('77', 16), int('d6', 16), int('26', 16), int(
        'e1', 16), int('69', 16), int('14', 16), int('63', 16), int('55', 16), int('21', 16), int('0c', 16), int('7d', 16)]
]

The functions below search in the S-Box/reverse S-Box and return the coordinates

In [88]:
def lookup(byte):
    x = byte >> 4
    y = byte & 15
    return sbox[x][y]

In [89]:
def reverse_lookup(byte):
    x = byte >> 4
    y = byte & 15
    return reverse_sbox[x][y]

##Implementing the Key Expansion

The next step is implementing the Key Expansion.
AES key expansion is the process of generating a set of round keys from the original encryption key for use in the AES algorithm. The key expansion algorithm takes as input the original key and produces a set of round keys, each of which is used in a specific round of the AES encryption or decryption process.

In [91]:
def expand_key(key, rounds):
    rcon = [[1, 0, 0, 0]]
    for i in range(1, rounds):
        rcon.append([rcon[-1][0]*2, 0, 0, 0])
        if rcon[-1][0] > 0x80:
            rcon[-1][0] ^= 0x11b

    key_matrix = break_into_blocks(key)[0]
    for round in range(rounds):
        last_col = []
        for row in key_matrix:
            last_col.append(row[-1])
        last_col_rotate_step = rotate_row_left(last_col)

        last_col_sbox_step = []
        for i in last_col_rotate_step:
          last_col_sbox_step.append(lookup(i))

        last_col_rcon_step = []
        for i in range(len(last_col_rotate_step)):
          last_col_rcon_step.append(last_col_sbox_step[i] ^ rcon[round][i])
        for r in range(4):
            key_matrix[r] += bytes([last_col_rcon_step[r] ^ key_matrix[r][round*4]])

        for i in range(len(key_matrix)):
            for j in range(1, 4):
                key_matrix[i] += bytes([key_matrix[i][round*4+j] ^ key_matrix[i][round*4+j+3]])
    return key_matrix

In [92]:
def extract_key_for_round(expanded_key, round):
    result = []
    for row in expanded_key:
        result.append(row[4*round: 4*(1+round)])
    return result

##Rotate Rows

The next important step is rotating the n-th row of block n times

In [93]:
def rotate_row_left(row, n=1):
  rotated_row = row[n:] + row[:n]
  return rotated_row

##Mix Columns

In AES the Mix Columns transformation is a step performed during the encryption process in each round. It operates on the columns of the state matrix to provide diffusion and further enhance the security of the cipher.

The MixColumns step operates on a 4x4 matrix, where each column is treated as a polynomial over the finite field. It involves multiplying each column by a fixed matrix transformation and reducing the result modulo a predefined polynomial.

The specific matrix used for this transformation is as follows:\
|2 3 1 1|\
|1 2 3 1|\
|1 1 2 3|\
|3 1 1 2|

In [94]:
def multiply_by_2(v):
  s = v << 1
  s &= 0xff
  if (v & 128) != 0:
    s = s ^ 0x1b
  return s

def multiply_by_3(v):
  return multiply_by_2(v) ^ v

In [95]:
def mix_column(column):
  new_matrix = [
      multiply_by_2(column[0]) ^ multiply_by_3(column[1]) ^ column[2] ^ column[3],
      multiply_by_2(column[1]) ^ multiply_by_3(column[2]) ^ column[3] ^ column[0],
      multiply_by_2(column[2]) ^ multiply_by_3(column[3]) ^ column[0] ^ column[1],
      multiply_by_2(column[3]) ^ multiply_by_3(column[0]) ^ column[1] ^ column[2]]
  return new_matrix
  
def mix_columns(matrix):
  new_matrix = [[],[],[],[]]
  for i in range(4):
      col = []
      for j in range(4):
        col.append(matrix[j][i])
      col = mix_column(col)
      for k in range(4):
          new_matrix[k].append(col[k])
  return new_matrix

##Add Sub Key

This step of encryption involves bitwise XOR operation between the state matrix and a round key.
Each byte of the round key is XORed with the corresponding byte in the state matrix.

In [96]:
def add_sub_key(block_matrix, key_matrix):
    new_matrix = []
    for i in range(4):
        row = []
        for j in range(4):
            row.append(block_matrix[i][j] ^ key_matrix[i][j])
        new_matrix.append(row)
    return new_matrix

##Initial and Final Rounds

We now have all the functions we need to get the main functions that will encode and decode the message

##Encryption

First, we supplement the data so that it can be divided into blocks. Then we divide the data into blocks of 16 bytes in the form of a 4*4 matrix

Then we expand the key for the multiple rounds, apply the original key to the blocks before start the rounds, and add sub key. For now on we will work with integers.

For AES-128 there are a total of 10 rounds. 
In each round we make byte substitution, shift rows, mix columns, and add sub key. However, in the last round, we don't mix the columns.

The result of this function is an encoded data

In [97]:
def encryption(key, data):
    pad = bytes(16 - len(data) % 16)
    if len(pad) != 16:
        data += pad
    matrices = break_into_blocks(data)

    expanded_key = expand_key(key, 11)

    temp_matrices = []
    round_key = extract_key_for_round(expanded_key, 0)

    for matrix in matrices:
        temp_matrices.append(add_sub_key(matrix, round_key))
    matrices = temp_matrices

# 1-9 round
    for round in range(1, 10):
        temp_matrices = []
        for matrix in matrices:
            sub_bytes_step = []
            for row in matrix:
                lst = []
                for val in row:
                    lst.append(lookup(val))
                sub_bytes_step.append(lst)

            shift_rows_step = []
            for i in range(4):
                shift_rows_step.append(rotate_row_left(sub_bytes_step[i], i))
            
            mix_column_step = mix_columns(shift_rows_step)
            round_key = extract_key_for_round(expanded_key, round)
            add_sub_key_step = add_sub_key(mix_column_step, round_key)
            temp_matrices.append(add_sub_key_step)
        matrices = temp_matrices

# 10-th round
    temp_matrices = []
    round_key = extract_key_for_round(expanded_key, 10)
    for matrix in matrices:
        sub_bytes_step = []
        for row in matrix:
          lst = []
          for value in row:
            lst.append(lookup(value))
          sub_bytes_step.append(lst)

        shift_rows_step = []
        for i in range(4):
            shift_rows_step.append(rotate_row_left(sub_bytes_step[i], i))

        add_sub_key_step = add_sub_key(shift_rows_step, round_key)
        temp_matrices.append(add_sub_key_step)
    matrices = temp_matrices

    int_stream = []
    for matrix in matrices:
        for column in range(4):
            for row in range(4):
                int_stream.append(matrix[row][column])
    return bytes(int_stream)

##Decryption

First, we break data into blocks. Then we undo 10-th round. To undo add_sub_key operation, (since it is XOR operation) we need to perform it again. To undo rotate_row(row, 1), we should apply rotate_row(row, -1). After this, we make byte substitution.

To undo 1-9 rounds we use same algorithm, except that we need to undo mix_columns operation. Doing the mix columns three times is equal to using the reverse matrix.

The next step is to reverse the first add sub key and transform matrix to bytes.

The result of this function is decoded data

In [98]:
def decryption(key, data):
    matrices = break_into_blocks(data)
    expanded_key = expand_key(key, 11)
    temp_matrices = []
    round_key = extract_key_for_round(expanded_key, 10)

# undo 10-th round
    temp_matrices = []

    for matrix in matrices:
        add_sub_key_step = add_sub_key(matrix, round_key)

        shift_rows_step = []
        for i in range(4):
            shift_rows_step.append(rotate_row_left(add_sub_key_step[i], -1 * i))
        sub_bytes_step = []
        for row in shift_rows_step:
            lst = []
            for value in row:
                lst.append(reverse_lookup(value))
            sub_bytes_step.append(lst)
        temp_matrices.append(sub_bytes_step)
    matrices = temp_matrices
# undo 1-9 round
    for round in range(9, 0, -1):
        temp_matrices = []
        for matrix in matrices:
            round_key = extract_key_for_round(expanded_key, round)
            add_sub_key_step = add_sub_key(matrix, round_key)
# undo mix_columns
            mix_column_step = mix_columns(add_sub_key_step)
            mix_column_step = mix_columns(mix_column_step)
            mix_column_step = mix_columns(mix_column_step)
            shift_rows_step = []
            for i in range(4):
                shift_rows_step.append(rotate_row_left(mix_column_step[i], -1 * i))
            sub_bytes_step = []
            for row in shift_rows_step:
                lst = []
                for value in row:
                    lst.append(reverse_lookup(value))
                sub_bytes_step.append(lst)
            temp_matrices.append(sub_bytes_step)
        matrices = temp_matrices
        temp_matrices = []

# Reversing the first add sub key
    round_key = extract_key_for_round(expanded_key, 0)
    for matrix in matrices:
        temp_matrices.append(add_sub_key(matrix, round_key))
    matrices = temp_matrices

# matrices back to bytes
    int_stream = []
    for matrix in matrices:
        for column in range(4):
            for row in range(4):
                int_stream.append(matrix[row][column])
    return bytes(int_stream)

#Test

Let's test this algorithm.\
First, we encode the message 'We love LA so much' using the key b'5ay2hkj02dfd83qu'.\
Then we decode the encoded message using the same key. \
As a result, we should get our original message



In [84]:
data =  b'We love LA so much'
key = b'5ay2hkj02dfd83qu'
encoded_data = encryption(key, data)

print("Data:", data, "\n\nEncoded data:", encoded_data)

decoded_data = decryption(key, encoded_data)

print("\nData:", encoded_data, "\n\nDecoded data:", decoded_data[:-(16 - len(data) % 16)])

Data: b'We love LA so much' 

Encoded data: b'\xd4\x19\xc9\x08\x1f\x92\x1a\x8b\x93\x144)\x1f\x02\xfd\xae/\xdc\xd8"{y4c\x8b\xdap\xf0Su\xb8\t'

Data: b'\xd4\x19\xc9\x08\x1f\x92\x1a\x8b\x93\x144)\x1f\x02\xfd\xae/\xdc\xd8"{y4c\x8b\xdap\xf0Su\xb8\t' 

Decoded data: b'We love LA so much'
